In [ ]:
!pip install pandas

In [ ]:
!pip install scikit-learn


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313472 sha256=13e8460184e8ba6b4732fb2d0901d336ba973fc59f8d4457cfbab9097545dbbd
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
!pip install NLTK

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import nltk
import fasttext
import string
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


nltk.download('punkt')
nltk.download('stopwords')

train_data = pd.read_csv("Train.csv")
test_data = pd.read_csv("Test.csv")
test_data = pd.read_csv("Valid.csv")

print(train_data.info())
print(test_data.info())

stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

train_data['preprocessed_text'] = train_data['text'].apply(preprocess_text)
test_data['preprocessed_text'] = test_data['text'].apply(preprocess_text)

tfidf_vectorizer = TfidfVectorizer(max_features=20000)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['text'])
y_train = train_data['label']
y_test = test_data['label']

tfidf_stop_vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
X_train_tfidf_stop = tfidf_stop_vectorizer.fit_transform(train_data['preprocessed_text'])
X_test_tfidf_stop = tfidf_stop_vectorizer.transform(test_data['preprocessed_text'])

lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, y_train)
y_pred_lr = lr_model.predict(X_test_tfidf)

print("Classification report Approach 1 (TF-IDF without stop words):")
print(classification_report(y_test, y_pred_lr, target_names=['Class 0', 'Class 1'], digits=4))

lr_stop_model = LogisticRegression()
lr_stop_model.fit(X_train_tfidf_stop, y_train)
y_pred_lr_stop = lr_stop_model.predict(X_test_tfidf_stop)

print("Classification report Approach 2 (TF-IDF with stop words removed):")
print(classification_report(y_test, y_pred_lr_stop, target_names=['Class 0', 'Class 1'], digits=4))

with open('train.txt', 'w', encoding='utf-8') as f:
    for label, text in zip(train_data['label'], train_data['text']):
        f.write(f"__label__{label} {text}\n")

with open('test.txt', 'w', encoding='utf-8') as f:
    for label, text in zip(test_data['label'], test_data['text']):
        f.write(f"__label__{label} {text}\n")

ft_model = fasttext.train_supervised(input='train.txt', epoch=25, lr=1.0, wordNgrams=2)
test_texts = test_data['text'].tolist()
predictions = [ft_model.predict(text)[0][0] for text in test_texts]


y_pred_ft = [1 if pred == '__label__1' else 0 for pred in predictions]
y_true_ft = ['__label__' + str(label) for label in y_test]

print("Classification report Approach 3 (FastText):")
print(classification_report(y_true_ft, predictions, target_names=['Class 0', 'Class 1'], digits=4))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   label   5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.3+ KB
None
Classification report Approach 1 (TF-IDF without stop words):
              precision    recall  f1-score   support

     Class 0     0.8999    0.8749    0.8872      2486
     Class 1     0.8796    0.9037    0.8915      2514

    accuracy                         0.8894      5000
   macro avg     0.8897    0.8893    0.8894      5000
weighted avg     0.8897    0.8894    0.8894      50